# Anytime Bottom-Up Rule Learning for Knowledge Graph Completion

Chúng tôi đề xuất một kỹ thuật *anytime bottom-up* để học các quy tắc logic từ các đồ thị tri thức lớn. Chúng tôi áp dụng các quy tắc đã học để dự đoán các ứng cử viên trong bối cảnh hoàn thành biểu đồ thị tri thức. Cách tiếp cận của chúng tôi vượt trội hơn so với các cách tiếp cận dựa trên quy tắc khác và nó cạnh tranh với tình trạng hiện đại, dựa trên các đại diện tiềm ẩn. Bên cạnh đó, cách tiếp cận của chúng tôi nhanh hơn đáng kể, đòi hỏi ít tài nguyên tính toán hơn và đưa ra lời giải thích về các quy tắc đề xuất một ứng cử viên.

## 1 Introduction

Hoàn thành đồ thị tri thức đã trở thành một lĩnh vực nghiên cứu sinh động trong vòng mười năm qua. Trong khi nghiên cứu hiện tại chủ yếu liên quan đến các biểu diễn ẩn dựa trên ý tưởng để nhúng đồ thị tri thức vào không gian vectơ chiều thấp, các phương pháp biểu tượng đã thu hút ít sự chú ý hơn [Wang et al., 2017]. Tuy nhiên, cách tiếp cận như vậy có một lợi thế lớn, đó là khả năng đưa ra lời giải thích theo các quy tắc *trigger a prediction* (kích hoạt dự đoán)

Trong bài báo này, chúng tôi đề xuất một kỹ thuật từ dưới lên để học hiệu quả các quy tắc logic từ các biểu đồ tri thức lớn. Công việc của chúng tôi được truyền cảm hứng từ cách tiếp cận học tập theo quy tắc từ dưới lên Golem [Muggleton và Feng, 1990] và Aleph [Srinivasan, 2000], đã được phát triển trong những ngày đầu của *Inductive Logic Programming* (ILP). Cách tiếp cận từ dưới lên dựa trên ý tưởng rằng một ví dụ là một biểu diễn nhỏ gọn của một quy tắc rất cụ thể có thể được khái quát hóa để nắm bắt một tập hợp con toàn diện của tất cả các **positive examples**. Aleph là một trong những cách tiếp cận như vậy. Nó biến đổi một *positive examples* nhất định thành quy tắc **Horn** được gọi là quy tắc dưới cùng. Quy tắc này sau đó được khái quát hóa bằng cách bỏ các *atoms* khỏi cơ thể quy tắc cho đến khi tìm thấy quy tắc đáp ứng tiêu chí chất lượng đã chọn. Các ví dụ được bao phủ bởi quy tắc được loại bỏ và cách tiếp cận được áp dụng lại cho đến khi tất cả các ví dụ được học

Cách tiếp cận của chúng tôi khác nhau theo nhiều cách. Trước hết, chúng ta không có một biên giới nghiêm ngặt cho chúng ta biết những quan sát nào thuộc về một ví dụ cụ thể và những gì không thuộc về nó. Thay vào đó, chúng ta phải quyết định những gì để coi là một ví dụ. Chúng tôi dựa trên khái niệm về một ví dụ về khái niệm đường dẫn. Theo nghĩa đó, cách tiếp cận của chúng tôi tương tự như thuật toán xếp hạng đường dẫn (PRA) [Lao et al., 2011]. Tuy nhiên, PRA coi các đặc trưng chỉ là một tập hợp con của các quy tắc mà chúng ta có thể tìm hiểu.

Thứ hai, chúng tôi quan tâm đến việc học các quy tắc không chắc chắn, tức là các quy tắc bao gồm ít nhất một số ví dụ đúng và thường là một số ví dụ sai. Ngay cả một quy tắc với độ tin cậy thấp vẫn có thể giúp chúng ta tạo ra thứ hạng ứng viên tốt hơn cho nhiệm vụ hoàn thành đồ thị tri thức. Hơn nữa, chúng tôi không thể xóa các ví dụ được quy định bởi một quy tắc, bởi vì có thể có các quy tắc khác cũng bao gồm (một số) các ví dụ này và các ví dụ khác, với điểm tin cậy khác nhau.

Thứ ba, thuật toán của chúng tôi được thiết kế để trở thành một công cụ khai thác quy tắc hiệu quả cho các đồ thị tri thức. Trong một biểu đồ tri thức, tất cả các sự kiện (còn được gọi là bộ ba) kết quả từ các vị từ nhị phân nối đất với các hằng số. Do đó, một đồ thị tri thức có thể được giải mã thành một tập hợp các đường dẫn có nhãn cạnh. Đây là lý do chính tại sao nó có ý nghĩa để tập trung vào các đường dẫn trong trường hợp không có các vị từ đơn vị hoặc các vị từ $n-ary$ với $n ≥ 3$.

Vấn đề hoàn thành biểu đồ tri thức (hoặc dự đoán liên kết) hiện đang bị chi phối bởi các phương pháp nhúng biểu đồ tri thức đã cho vào một không gian đặc trưng ẩn. Học một **đại diện biểu tượng** rõ ràng hiếm khi được đề xuất thay thế. Điều này có thể liên quan đến giả định cơ bản rằng một cách tiếp cận dựa trên quy tắc không thể giải quyết nhiều hơn chỉ là một tập hợp con tầm thường. Xem, ví dụ như cuộc thảo luận về mô hình nghịch đảo được đề xuất trong [Dettmers et al., 2018] và các nhà phê bình liên quan đến sự dư thừa trong FB15k [Toutanova và Chen, 2015]. Một giả định khác có thể là việc học quy tắc không thể được áp dụng cho các bộ dữ liệu lớn. Vì lý do đó, nghiên cứu hiện tại có liên quan đến sự kết hợp của các quy tắc và nhúng (ví dụ: [Guo et al., 2018]). Chúng tôi lập luận rằng các giả định cơ bản là sai và đưa ra kết quả hỗ trợ cho yêu cầu của chúng tôi. Cụ thể, chúng tôi đề xuất một thuật toán từ dưới lên bất cứ lúc nào để học các quy tắc và áp dụng phương pháp của chúng tôi vào nhiệm vụ hoàn thành kiến ​​thức. Chúng tôi trình bày kết quả cho năm bộ dữ liệu khác nhau. Ba trong số chúng đã được đề xuất là trường hợp khó cho các phương pháp đơn giản (dựa trên quy tắc) tận dụng các đối xứng và các dự phòng khác. Cách tiếp cận của chúng tôi thực hiện tốt như và đôi khi tốt hơn hầu hết các mô hình đã được đề xuất gần đây. Kết quả của phương pháp của chúng tôi vẫn rất tốt nếu chúng tôi dừng thuật toán đã có sau một khoảng thời gian ngắn. Hơn nữa, các tài nguyên cần thiết về bộ nhớ và thời gian chạy nhỏ hơn đáng kể so với các tài nguyên được yêu cầu bởi các phương pháp dựa trên đặc trưng ẩn.

## 5 Experiments

Chúng tôi đã sử dụng trong các thử nghiệm của mình bộ dữ liệu FB15 (k), biến thể FB15-237, WN18, biến thể WN18RR đã sửa đổi của nó và YAGO03-10 (viết tắt là YAGO). Các bộ dữ liệu FB (WN) dựa trên tập hợp con FreeBase (WordNet). FB15 và WN18 đã được sử dụng lần đầu tiên trong [Bordes et al., 2013]. Họ đã bị chỉ trích trong một số bài báo: do dư thừa, một phần lớn các mẫu thử có thể được giải quyết bằng cách khai thác các quy tắc khá đơn giản. Cả FB15-237 [Toutanova và Chen, 2015] và WN18RR [Dettmers et al., 2018] đã được đề xuất là các biến thể sửa đổi trong đó các khoản dự phòng đã bị loại bỏ. FB15-237 thậm chí còn tiến thêm một bước. Nó không chứa các trường hợp thử nghiệm có thể được giải quyết bằng các quy tắc C với một nguyên tử cơ thể, mặc dù tập huấn luyện cho thấy các quy tắc tạo ra các quy tắc này. Điều này làm cho FB15-237 hơi phi thực tế và là một thử nghiệm khó khăn cho cách tiếp cận dựa trên quy tắc. Bộ dữ liệu YAGO đã được sử dụng trong [Dettmers et al., 2018] dưới dạng tập dữ liệu bổ sung. Tổng quan về các bộ dữ liệu được đưa ra trong Bảng 1. Các số được báo cáo trong ba dòng đầu tiên đề cập đến tập huấn luyện. Dòng cuối cùng đề cập đến số lượng bộ ba trong bộ thử nghiệm. Mỗi bộ ba thử nghiệm có thể được chia thành hai trường hợp thử nghiệm. Xét về bộ ba và thực thể (hằng số) YAGO03-10 là tập dữ liệu lớn nhất.

Như trong [Bordes et al., 2013] chúng tôi tính toán các lần truy cập được lọc @ 1 và các lần truy cập được lọc @ 10. Sau đây chúng tôi chỉ đơn giản đề cập đến các giá trị này mà không có tính từ Bộ lọc được lọc. Chúng tôi không tính toán chính xác MRR được lọc (có nghĩa là đối ứng xếp hạng-**mean rank reciprocal**), vì cách tiếp cận của chúng tôi không được thiết kế để tính toán thứ hạng hoàn chỉnh mà chỉ xếp hạng top-k. Thay vì tính toán giá trị chính xác, chúng tôi giả định rằng bất kỳ ứng cử viên nào được xếp hạng ở vị trí> k không phải là một dự đoán chính xác. Điều này dẫn đến giới hạn dưới của MRR được lọc, mà chúng tôi trình bày trong Bảng 2 có tiền tố ≥. Giá trị chính xác thường cao hơn một chút. Chúng tôi đã đặt tên cho phương pháp tiếp cận của chúng tôi là AnyburL (Học quy tắc từ dưới lên bất cứ lúc nào). AnykinaL được viết bằng Java và không yêu cầu các thư viện bên ngoài. Mã nguồn và bộ dữ liệu, được sử dụng trong các thử nghiệm, có thể được tìm thấy tại http: // web. informationatik.uni-mannheim.de/AnykinaL/. Chúng tôi đã tiến hành tất cả các thử nghiệm của mình trên một máy ảo với 4 lõi (mỗi lõi 2400 MHz) và RAM 16 GB.

Chúng tôi chưa sử dụng các bộ xác thực để tìm cài đặt tham số cụ thể cho tập dữ liệu. Đối với tất cả các thử nghiệm của chúng tôi, chúng tôi đã sử dụng chính xác cùng một cài đặt tham số. Chúng tôi đã chọn tiêu chí chất lượng Q để chỉ cho phép những quy tắc tạo ra ít nhất hai dự đoán chính xác, đó là tiêu chí rất lỏng lẻo. Chúng tôi đã đặt tỷ lệ bão hòa yêu cầu sat đến 99%. Chúng tôi đặt ts = 1 giây, pc = 5 và s = ​​500, đây là cỡ mẫu xác định độ chính xác của giá trị độ tin cậy. Như cài đặt mặc định, chúng tôi đã sử dụng chiến lược tổng hợp tối đa. Chúng tôi cũng đã tiến hành một thử nghiệm trong đó chúng tôi so sánh chiến lược này với tập hợp Không gây ồn ào. Chúng tôi đã thực hiện hai sửa đổi thêm. Việc triển khai của chúng tôi xen kẽ giữa các khoảng thời gian trong đó chúng tôi chỉ lấy mẫu các đường dẫn tuần hoàn; và khoảng thời gian nơi chúng ta lấy mẫu tất cả các đường dẫn có thể. Điều này cho phép sử dụng một thuật toán tìm chu kỳ hiệu quả hơn. Hơn nữa, chúng tôi đã dừng tìm kiếm một cách rõ ràng các đường dẫn tuần hoàn một khi chúng tôi đạt đến độ bão hòa được xác định cho các quy tắc C có độ dài 3, bởi vì các quy tắc tuần hoàn dài hơn có thể rất tốn kém về thời gian chạy.

### 5.1 Anytime Behavior and Runtime
Chúng ta phải phân biệt giữa thời gian cần thiết để tìm hiểu các quy tắc và thời gian áp dụng chúng để giải quyết công việc. Rõ ràng, chúng ta càng dành nhiều thời gian cho việc học, chúng ta càng học được nhiều quy tắc hơn; và chúng ta càng học nhiều quy tắc, thời gian cần thiết để áp dụng các quy tắc này sẽ càng lâu. Kết quả lần $Hits@1$ và kết quả lần $hits@10$ được chỉ định cho tất cả năm bộ dữ liệu trong Bảng 2 dựa trên các bộ quy tắc đã được học sau $10, 100, 1000$ và $10000$ giây. Đối với tất cả các bộ dữ liệu, chúng tôi quan sát một mô hình tương tự. Kết quả đã tốt đáng ngạc nhiên sau một giai đoạn học tập $10$ giây. Đối với các bộ dữ liệu nhỏ hơn ($WN18$ và $WN18RR$), các kết quả này đã tốt như trạng thái hiện tại của kết quả hiện đại. Khi chúng tôi tìm hiểu các quy tắc trong $100$ và $1000$ giây (chưa đến nửa giờ), kết quả sẽ được cải thiện hơn nữa và sự cải thiện này cao hơn đối với các bộ dữ liệu lớn hơn.

**Learning rules** trong một thời gian dài ($10000$ giây) có thể có tác động hơi tiêu cực đến kết quả. Điều này đặc biệt là trường hợp cho các bộ dữ liệu WN18 và FB15. Các bộ dữ liệu này bị chi phối bởi các liên kết dự phòng được phản ánh trong các quy tắc $C$ có độ dài $1$. Các quy tắc này có thể được tìm thấy nhanh chóng ngay từ đầu. Nếu chúng ta học lâu hơn, có thể xảy ra rằng có một số quy tắc rất cụ thể trong số rất lớn các quy tắc đã học tạo ra nhiễu. Tuy nhiên, xu hướng này không tiếp tục nếu chúng ta chạy công cụ lâu hơn nữa. Chúng tôi cũng đã chạy AnyburL trên WN18 và FB15 trong $20000$ giây và gần như không có sự khác biệt so với kết quả được báo cáo cho lần chạy $10000 $giây (ví dụ, WN18 thay đổi từ 95,42 thành 95,40).

Đối với YAGO, chúng tôi cho thấy (i) tác động của thời gian học đến chất lượng kết quả (trục y ở bên trái), (ii) thời gian cần thiết để áp dụng các quy tắc đã học (chiếu cho trục x) và (iii) số quy tắc đã được học (đường màu tím, trục y ở bên phải). Thời gian ứng dụng, tham chiếu đến bộ thử nghiệm đầy đủ, được hiển thị dưới dạng hình chiếu cho trục x. Ví dụ: học các quy tắc trong 10 giây, kết quả là số lần truy cập @ 10 là 55,3% và một bộ quy tắc bao gồm 4531 quy tắc. Nó đòi hỏi 42 giây để áp dụng quy tắc này được đặt cho 5000 trường hợp thử nghiệm của bộ dữ liệu. Lưu ý rằng trục x và trục y ở bên phải sử dụng thang đo logarit.

Hình 2 cho thấy AnykinaL bắt đầu ở mức cao và cải thiện kết quả về mặt hàm logarit. Tập hợp các quy tắc đã học dường như phát triển gần như tuyến tính với mức giảm nhẹ khi chúng ta chạy AnyburL lâu hơn. AnyburL đạt đến độ bão hòa 0,99 cho quy tắc C có độ dài 3 sau 84 giây, trong khi độ bão hòa cho quy tắc AC1 và AC2 có độ dài 1 không đạt được trong khoảng thời gian chung là 30000 giây. Điều này rõ ràng là do số lượng hằng số cao trong bộ dữ liệu YAGO. Điều này chỉ ra rằng các kết quả tốt đạt được trong thời gian ngắn chủ yếu dựa trên các quy tắc C, trong khi các cải tiến nhỏ nhưng ổn định tuân theo các quy tắc AC1 và AC2.

Thời gian áp dụng các quy tắc dành cho các bộ quy tắc nhỏ cao hơn thời gian cần thiết để tìm hiểu chúng trên toàn bộ bộ kiểm tra. Tuy nhiên, nếu các quy tắc được đặt lớn hơn thì cần ít thời gian hơn để dự đoán. Một bộ quy tắc được học trong 1000 giây, cũng cần khoảng 1000 giây để được áp dụng. Đối với các quy tắc lớn hơn, thời gian học chiếm ưu thế trong thời gian áp dụng. Cũng lưu ý rằng thời gian ứng dụng nhỏ hơn đối với bốn bộ dữ liệu khác.



Thời gian áp dụng các quy tắc dành cho các bộ quy tắc nhỏ cao hơn thời gian cần thiết để tìm hiểu chúng trên toàn bộ bộ kiểm tra. Tuy nhiên, nếu các quy tắc được đặt lớn hơn thì cần ít thời gian hơn để dự đoán. Một bộ quy tắc được học trong 1000 giây, cũng cần khoảng 1000 giây để được áp dụng. Đối với các quy tắc lớn hơn, thời gian học chiếm ưu thế trong thời gian áp dụng. Cũng lưu ý rằng thời gian ứng dụng nhỏ hơn đối với bốn bộ dữ liệu khác.